# **Setup**

In [ ]:
import os
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import math as math
import glob as glob
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import plot, iplot, init_notebook_mode

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

# **Data Processing**

## Loading and Importing Data

In [ ]:
# Link google drive for model and data
from google.colab import drive
drive.mount('/content/drive')

#### Read File In

Data used to simulate predictions will still be the data from weather stations. 

However ideally, these data would be collected from the hardware devices and transmitted to this server (computer) running the AI model prediction through serial connections.

Depending on the format that hardware collected data is, some variables etc. might need to be renamed but the general structure of the script should remain the same.

Comment: Assume the csv file is in correct format. The truth is I manually combined the five stations' weather data properly.

###Read file in with proper date

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/LydiaSS/Solar_Irradiance_Forecast_RNN/main/ACISHourlyData_GoodOrder.csv", encoding='utf-8', index_col=None, header=0, thousands=',')


df['Incoming Solar Rad. (W/m2)'] = pd.to_numeric(df['Incoming Solar Rad. (W/m2)'], downcast="float")

def file_cleanup(df):
    # Some Cleaning
    df.drop_duplicates()
    # df = df[df['Incoming Solar Rad. (W/m2)'].notna()]

    # Get rid of columns that don't contain any useful data (like comment cols)
    preliminary_column_mask = [
          'Station Name',
          'Date (Local Standard Time)', 
          'Air Temp. Inst. (C)',
          'Air Temp. Min. (C)',
          'Air Temp. Max. (C)',
          'Air Temp. Avg. (C)',
          'Humidity Inst. (%)',
          'Humidity Avg. (%)',
          'Incoming Solar Rad. (W/m2)',
          'Precip. Accumulated (mm)',
          'Precip. (mm)',
          'Wind Speed 10 m Syno. (km/h)',
          'Wind Dir. 10 m Syno. ()',
          'Wind Speed 10 m Avg. (km/h)',
          'Wind Dir. 10 m Avg. ()',]

    df = df[df.columns.intersection(preliminary_column_mask)]

    # Rename
    df = df.rename(columns={'Wind Dir. 10 m Syno. ()': 'Wind Dir. 10 m Syno. (deg)', 
                      'Wind Dir. 10 m Avg. ()': 'Wind Dir. 10 m Avg. (deg)'})

    df.head()

    return df

df = file_cleanup(df)
df

In [ ]:
# Separate the data by station location
#grouped = df.groupby(df['Station Name'])

df_SOUTH_CAMPUS = df.loc[df['Station Name'].isin(["Edmonton South Campus UA"])]
df_ST_ALBERT = df.loc[df['Station Name'].isin(["St. Albert Research"])]

df_OLIVER = df.loc[df['Station Name'].isin(["Oliver AGDM"])]
df_LEGAL = df.loc[df['Station Name'].isin(["Legal AGCM"])]
df_ST_FRANCIS = df.loc[df['Station Name'].isin(["St. Francis AGCM"])]

station_dfs = [df_SOUTH_CAMPUS, df_ST_ALBERT, df_OLIVER, df_LEGAL, df_ST_FRANCIS]
# Include weather station geographical location data, note the Lat and Long also have a direction associated but since these
#   stations are close to each other the directions are the same and  not considered in this application
# Lat and Long data processing: data are taken from the gov of Canada website and converted to decimal using https://www.fcc.gov/media/radio/dms-decimal
station_locations = [
    {
      "Name": "Edmonton South Campus UA",
      "Latitude": 53.490014,
      "Longitude": 113.537778,
      "Elevation (m)": 670.00,
      "df": df_SOUTH_CAMPUS,
    },
    {
      "Name": "St. Albert Research",
      "Latitude": 53.691944,
      "Longitude": 113.619722,
      "Elevation (m)": 687.00,
      "df": df_ST_ALBERT,
    },
    {
      "Name": "Oliver AGDM",
      "Latitude": 53.65,
      "Longitude": 113.35,
      "Elevation (m)": 665.00,
      "df": df_OLIVER,
    },
    {
      "Name": "Legal AGCM",
      "Latitude": 54.003056,
      "Longitude": 113.474445,
      "Elevation (m)": 680.00,
      "df": df_LEGAL,
    },
    {
      "Name": "St. Francis AGCM",
      "Latitude": 53.294731,
      "Longitude": 114.320011,
      "Elevation (m)": 809.00,
      "df": df_ST_FRANCIS,
    },    
]

In [ ]:
df_LEGAL["date"] = pd.to_datetime(df['Date (Local Standard Time)'])
# I realized that one location doesn't have solar irradiance data before the following date
res = df_LEGAL[~(df_LEGAL['date'] < '2018-07-01')]

# Let's cut all dataframe to ^ date then
for i in range(5):
  station_dfs[i]["date"] = pd.to_datetime(df['Date (Local Standard Time)'])
  station_dfs[i] = station_dfs[i][~(station_dfs[i]['date'] < '2018-07-01')]

# Take out timestamps after 10pm and before 6am for all 
for i in range(5):
  # Get all wanted column names from df
  station_dfs[i] = station_dfs[i].set_index('date')
  station_dfs[i] = station_dfs[i].between_time("5:00", "22:00")

## Process Data

In [ ]:
# For ease of switching between data files
ws_col = 'Wind Speed 10 m Avg. (km/h)'
ws_max_col = 'Wind Speed 10 m Syno. (km/h)'
wd_col = 'Wind Dir. 10 m Avg. (deg)'
date_col = 'Date and Time'
temp_col = 'Air Temp. Avg. (C)'
solar_irr_col = 'Incoming Solar Rad. (W/m2)'
humidity_col = 'Humidity Avg. (%)'

# Create a list to hold training only columns (got rid of min and max)
train_col_mask = [date_col, temp_col, solar_irr_col]

# Save timestamp into its own df first
datetime = pd.to_datetime(station_dfs[0].reset_index().pop('Date (Local Standard Time)'), format='%Y-%m-%d %H:%M')
timestamp_s = datetime.map(pd.Timestamp.timestamp)

# Reset index so index of df starts at 0
station_dfs[0] = station_dfs[0].reset_index()

for i in range(5):
  # Get all wanted column names from df
  station_dfs[i] = station_dfs[i][station_dfs[i].columns.intersection(train_col_mask)]
  # Equal all the index
  station_dfs[i].index = station_dfs[0].index


# Combine
df_all = pd.concat(station_dfs, axis = 1)

### Time Data

In [ ]:
# For periodicity within a day, we find the total number of seconds in a day
day = 24*60*60
# For periodicity within a year, we find the total number of seconds in a year
year = (365.2425)*day

df_all['Day sin'] = np.sin(timestamp_s * (2 * np.pi / day))
# Apparently cos has worse correlation with the solar irradiance lol
df_all['Day cos'] = np.cos(timestamp_s * (2 * np.pi / day))
df_all['Year sin'] = np.sin(timestamp_s * (2 * np.pi / year))
df_all['Year cos'] = np.cos(timestamp_s * (2 * np.pi / year))

plt.figure(0)
plt.plot(np.array(df_all['Day sin'])[:18*2])
plt.plot(np.array(df_all['Day cos'])[:18*2])
plt.xlabel('Time [h]')
plt.title('Time of day signal')

plt.figure(1)
plt.plot(np.array(df_all['Year sin'])[:math.floor(year/60**2)])
plt.plot(np.array(df_all['Year cos'])[:math.floor(year/60**2)])
plt.xlabel('Time [day]')
plt.title('Time of year signal')

In [ ]:
# Now we drop NA after adding time back
df_all = df_all.dropna()
df_all

### Normalize the data

In [ ]:
# Normalize the data the same way as in training
temp_mean = 3.686254
temp_std = 12.321960
solar_irr_max = 905.0
year_sin_mean = -0.009238
year_sin_std = 0.716370
year_cos_mean = 0.036401
year_cos_std = 0.696765
day_sin_mean = -0.114835
day_sin_std = 0.750117
day_cos_mean = -0.278348
day_cos_std = 0.588838

# Calculate mean and std
mean = [temp_mean, 0, temp_mean, 0, temp_mean, 0, temp_mean, 0, temp_mean, 0, day_sin_mean, day_cos_mean, year_sin_mean, year_cos_mean]
std = [temp_std, 1, temp_std, 1, temp_std, 1, temp_std, 1, temp_std, 1, day_sin_std, day_cos_std, year_sin_std, year_cos_std]


In [ ]:
# Normalize, note solar irradiance is normalized differently
solar_irr_normalized = df_all[solar_irr_col]/solar_irr_max

df_all_normalized = (df_all - mean) / std
df_all_normalized[solar_irr_col] = solar_irr_normalized
df_all_normalized.describe().transpose()

In [ ]:
# Sanity check with graphs
train_std = df_all_normalized.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=train_std)
_ = ax.set_xticklabels(df_all_normalized.keys(), rotation=90)

## Data windowing

Data windowing needs a lot of changes because we have multiple dataframes to generate windows from.

In [ ]:
class WindowGenerator():
  def __init__(self, input_width, label_width, shift,
               df=df_all_normalized,
               label_columns=None):
    # Store the raw data.
    self.train_df = df_all_normalized

    # Work out the label column indices.
    self.label_columns = label_columns
    if label_columns is not None:
      self.label_columns_indices = {name: i for i, name in
                                    enumerate(label_columns)}
    self.column_indices = {name: i for i, name in
                           enumerate(df_all_normalized.columns)}

    # Work out the window parameters.
    self.input_width = input_width
    self.label_width = label_width
    self.shift = shift

    self.total_window_size = input_width + shift

    self.input_slice = slice(0, input_width)
    self.input_indices = np.arange(self.total_window_size)[self.input_slice]

    self.label_start = self.total_window_size - self.label_width
    self.labels_slice = slice(self.label_start, None)
    self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

  def __repr__(self):
    return '\n'.join([
        f'Total window size: {self.total_window_size}',
        f'Input indices: {self.input_indices}',
        f'Label indices: {self.label_indices}',
        f'Label column name(s): {self.label_columns}'])

In [ ]:
def split_window(self, features):
  inputs = features[:, self.input_slice, :]
  labels = features[:, self.labels_slice, :]
  if self.label_columns is not None:
    labels = tf.stack(
        [labels[:, :, self.column_indices[name]] for name in self.label_columns],
        axis=-1)

  # Slicing doesn't preserve static shape information, so set the shapes
  # manually. This way the `tf.data.Datasets` are easier to inspect.
  inputs.set_shape([None, self.input_width, None])
  labels.set_shape([None, self.label_width, None])

  return inputs, labels

WindowGenerator.split_window = split_window

### Plot the resulting windows

In [ ]:
def plot(self, model=None, plot_col=solar_irr_col, max_subplots=3, test=False):
  '''
  This function plots the window showing inputs vs labels.
  This function plots the prediction vs expected if a model is given, however in this case the label_indices
    must match the number of elements in predictions. Double check when creating the window. length of prediction = length of input_indices
  '''
  if test:
    inputs, labels = next(iter(self.train))
  else:
    inputs, labels = self.example

  plt.figure(figsize=(12, 8))
  plot_col_index = self.column_indices[plot_col]
  max_n = min(max_subplots, len(inputs))
  for n in range(max_n):
    plt.subplot(max_n, 1, n+1)
    plt.ylabel(f'{plot_col} [normed]')
    plt.plot(self.input_indices, inputs[n, :, plot_col_index],
             label='Inputs', marker='.', zorder=-10)

    if self.label_columns:
      label_col_index = self.label_columns_indices.get(plot_col, None)
    else:
      label_col_index = plot_col_index

    if label_col_index is None:
      continue

    plt.scatter(self.label_indices, labels[n, :, label_col_index],
                edgecolors='k', label='Labels', c='#2ca02c', s=64)
    if model is not None:
      predictions = model(inputs)
      plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

WindowGenerator.plot = plot

### Convert Windows to Dataset Pairs

```make_dataset``` method will take a time series DataFrame and convert it to a ```tf.data.Dataset``` of (input_window, label_window) pairs using the ```preprocessing.timeseries_dataset_from_array``` function. Converting to tf Dataset will allow access to functions like repeat etc. as well.

In [ ]:
def make_dataset(self, data, sequence_stride=18):
  data = np.array(data, dtype=np.float32)
  # This function splits a dataset into a sliding time series window
  # https://www.tensorflow.org/api_docs/python/tf/keras/utils/timeseries_dataset_from_array
  ds = tf.keras.preprocessing.timeseries_dataset_from_array(
      data=data,
      targets=None,
      sequence_length=self.total_window_size,
      sequence_stride=sequence_stride,
      # The shuffle can be set to False here for the dataset to be in chronological order
      shuffle=True,
      batch_size=1,)

  ds = ds.map(self.split_window)

  return ds

WindowGenerator.make_dataset = make_dataset

In [ ]:
@property
def train(self):
  return self.make_dataset(self.train_df)

@property
def example(self):
  """Get and cache an example batch of `inputs, labels` for plotting."""
  # Lydia: I added the following line of code
  #   This ensures that everytime WindowGenerator.example is called, we can get a new iteration
  #   of result from the test dataset.
  """Get and cache an example batch of `inputs, labels` for plotting."""
  result = getattr(self, '_example', None)
  if result is None:
    # No example batch was found, so get one from the `.train` dataset
    result = next(iter(self.train))
    # And cache it for next time
    self._example = result
  return result

WindowGenerator.train = train
WindowGenerator.example = example

# Single Output Model

In [ ]:
# Load Model
model_name = 'model5'
lstm_model = tf.keras.models.load_model(model_name)
lstm_model.summary()

# Convert to Solar Irradiance

Obtain the prediction result of at a particular time.

For demonstration purpose let's just try to grab the last frame from the dataset for now.

In [ ]:
# convert an array of values into a dataset matrix
def create_dataset(dataset, look_back=0):
	dataX = []
	for i in range(len(dataset)-look_back):
		a = dataset[i:(i+look_back+1)]
		dataX.append(a)
	return np.array(dataX)
 
def reshape_dataset(dataset, samples=1, steps=18, columns=14):
  dataX = np.reshape(dataset,(samples, steps, columns))
  return dataX

def get_data_frame_by_date(date, df_pred, df_actual, window_size):
  required_time = pd.to_datetime(date, format='%Y-%m-%d %H:%M')

  index = datetime.to_list().index(required_time)

  if index <= window_size:
    print("Does not have enough data before the specified time to make predictions.")

  prediction = df_pred.iloc[index-window_size: index]

  theoretical = df_actual.iloc[index-window_size:index+1]

  return prediction, theoretical



def unormalize_predictions(prediction, mean, std):
  # For current normalization method, there should not be data below 0
  for i in range(len(prediction[0])):
    number = prediction[0][i]
    if number < 0:
      prediction[0][i] = 0
  prediction = prediction*std+mean
  return prediction


In [ ]:
def plot_predictions(predictions, samples=1, steps=18, columns=14, plot_col_index = 0, theoretical=None):
  plt.figure(figsize=(12, 8))
  theoretical_indices = range(1, len(theoretical)+1)
  label_indices = range(2, steps+2)
  
  for n in range(samples):
    plt.subplot(samples, 1, n+1)
    plt.ylabel(f'Solar Irradiance')

    if samples == 1:
      plt.plot(theoretical_indices, theoretical.iloc[:, plot_col_index],
              label='Inputs', marker='.', zorder=-10)
      plt.scatter(theoretical_indices, theoretical.iloc[:, plot_col_index],
                  edgecolors='k', label='Labels', c='#2ca02c', s=64)
    else:
      plt.plot(theoretical_indices, theoretical.iloc[n, :, plot_col_index],
              label='Inputs', marker='.', zorder=-10)
      plt.scatter(theoretical_indices, theoretical.iloc[n, :, plot_col_index],
                  edgecolors='k', label='Labels', c='#2ca02c', s=64)
      
    # Plot predictions
    plt.scatter(label_indices, predictions[n, :],
                  marker='X', edgecolors='k', label='Predictions',
                  c='#ff7f0e', s=64)

    if n == 0:
      plt.legend()

  plt.xlabel('Time [h]')

In [ ]:
def interactive_plot_predictions(predictions, theoretical, steps=18, columns=14, plot_col = solar_irr_col):
  # Create indices
  # TODO: make this hour relevant
  theoretical_indices = range(1, len(theoretical)+1)
  label_indices = range(2, steps+2)

  # Create dataframe for predictions to use for px
  df_pred = pd.DataFrame(predictions[0, :], columns = [plot_col])
  df_pred['index'] = label_indices

  # Create dataframe for theoretical data
  df_theory = pd.DataFrame(theoretical.iloc[:, 1], columns = [plot_col])
  df_theory['index'] = theoretical_indices

  # fig = px.line(df_theory, x="index", y=plot_col, markers=True)
  # fig.show() 
  # # Plot predictions
  # fig.add_scatter(df_pred, x="index", y=plot_col)

  trace1 = go.Scatter(
      x=df_theory["index"], y=df_theory[plot_col],
      name='Measured Solar Irr',
  )
  
  trace2 = go.Scatter(
      x=df_pred["index"], y=df_pred[plot_col],
      name='Predicted Solar Irr',
  )

  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(trace1)
  fig.add_trace(trace2,secondary_y=True)
  fig['layout'].update(height = 600, width = 800, title = 'Solar Irradiance Prediction VS Expected' ,xaxis=dict(
      tickangle=-90
    ))
  fig.show()
  return fig

In [ ]:
# Get data from csv
look_back = 0
sample = 1
step = 18
column = 14
total_window_size = 18

date = "2020-04-01 22:00"

data, theoretical_data = get_data_frame_by_date(date, df_all_normalized, df_all, total_window_size)

# reshape input to be [samples, time steps, features]
dataX = reshape_dataset(create_dataset(data, look_back), sample, step, column)

# Making Predictions
predictions = lstm_model.predict(dataX)

true_pred = unormalize_predictions(predictions, 0, solar_irr_max)

fig = interactive_plot_predictions(true_pred, theoretical_data)

### Plot

In [ ]:
#print(true_pred[0:,])
fig = interactive_plot_predictions(true_pred, theoretical_data)

# # Turn on interactive mode
# plt.ion()

# Dash

In [ ]:
pip install -q dash==1.19.0

In [ ]:
pip install -q jupyter_dash==0.3.0

In [ ]:
pip install -q dash-cytoscape

In [ ]:
from jupyter_dash import JupyterDash  # pip install dash
import dash_cytoscape as cyto  # pip install dash-cytoscape==0.2.0 or higher
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output, State
import pandas as pd  # pip install pandas
import plotly.express as px
import math
from dash import no_update


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [ ]:
colors = {
    'background': '#ffffff',
    'line': '#ebebeb',
    'text': '#000000',
}

# Get data from csv
look_back = 0
sample = 1
step = 18
column = 14
total_window_size = 18

fig.update_layout(
    plot_bgcolor=colors['line'],
    paper_bgcolor=colors['background'],
    font_color=colors['text']
)

app.layout = html.Div(style={'backgroundColor': colors['background'], 'margin':'auto', 'width': "50%"},
    children=[
                        
      html.H1(children='Multi-location Solar Irradiance Forecast',
              style={'textAlign': 'center',
                     'color': colors['text']}
      ),

      html.Div(children='''Group 003''', 
          style={'textAlign': 'center',
                 'color': colors['text']}
      ),

      html.Div([
        "Date and Hour (yyyy-mm-dd hh:00):  ",
        dcc.Input(id='date-input', value='2020-04-01 22:00', type='text'),
        # Submit button
        html.Button(id='submit-button-state', n_clicks=0, children='Enter'),

        dcc.Graph(
            id='Forecast',
            figure=fig,
            style={"display": "block",
                   
                   "margin-left": "auto",
                   "margin-right": "auto",
            },
        ),
      ], style = {'textAlign': 'center', 'justify-content': 'center'}),
      html.Br(),
      html.Div(id='solar_irr_output'),
    ]
)


# Plot update call back
@app.callback(
    Output(component_id='Forecast', component_property='figure'),
    # Output(component_id='solar_irr_output', component_property='children'),
    Input(component_id='date-input', component_property='value')
)

# Figure update function
def update_figure(datetime):  
  data, theoretical_data = get_data_frame_by_date(datetime, df_all_normalized, df_all, total_window_size)

  # reshape input to be [samples, time steps, features]
  dataX = reshape_dataset(create_dataset(data, look_back), sample, step, column)

  # Making Predictions
  predictions = lstm_model.predict(dataX)

  true_pred = unormalize_predictions(predictions, 0, solar_irr_max)

  fig = interactive_plot_predictions(true_pred, theoretical_data)

  fig.update_layout(transition_duration=500)

  return fig

if __name__ =='__main__':
    app.run_server(mode="external", debug=True)